In [ ]:
import os

import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px

from SMPLmeasure import SMPLmeasure

In [ ]:
df_ground_truth = pd.read_csv("./examples/ground_truth.csv").set_index("info")
df_ground_truth_partial = df_ground_truth.iloc[2:8, :].reset_index(drop=True)
ground_truth_dict = df_ground_truth.to_dict()

anonimos = [
    "anonimo1",
    "anonimo2",
    "anonimo3",
    "anonimo4",
    "anonimo5",
    "anonimo6",
    "anonimo7",
    "anonimo8",
    "anonimo9",
    "anonimo10",
    "anonimo11",
    "anonimo12",
    "anonimo13",
    "anonimo14",
    "anonimo15",
    "anonimo16",
    "anonimo17",
    "anonimo18",
    "anonimo19",
    "anonimo20",
    "anonimo21",
]

measure_smpl = SMPLmeasure.SMPLmeasure()

df_results = []

for anonimo in anonimos:

    samples = os.listdir(f"./examples/{anonimo}/")
    for sample in samples:
        print(anonimo, sample)
        try:
            result_path = f"./results/{anonimo}_{sample}/"

            result = measure_smpl.measure_smpl(
                filename=result_path + "front_pred.ply", height=float(ground_truth_dict[anonimo]["height"])
            )

            df = pd.DataFrame({
                "anonimo": [anonimo] * 6,
                "sample": [sample] * 6,
                "body_part": ["upper_arm", "chest", "waist", "hip", "thigh", "calf"],
                "value": [
                    np.mean([result["left_arm"], result["right_arm"]]),
                    result["chest"],
                    result["waist"],
                    result["hip"],
                    np.mean([result["left_thigh"], result["right_thigh"]]),
                    np.mean([result["left_calf "], result["right_calf"]]),
                ],
            })

            df["ground_truth"] = df_ground_truth_partial[anonimo].astype("float")

            df_results.append(df)
        except:
            print("Error")


df_results = pd.concat(df_results)

In [ ]:
df_results["error"] = 100 * np.abs(df_results["value"] - df_results["ground_truth"]) / df_results["ground_truth"]

In [ ]:
data = df_results["error"]

print(f"Mean error: {np.round(np.mean(data),2)}%")

bin_size = 5

fig = go.Figure()

fig.add_trace(
    go.Histogram(x=data, xbins=dict(start=min(data), end=max(data), size=bin_size), autobinx=False, histnorm="percent")
)

fig.update_layout(
    xaxis_title="Errore Percentuale",
    yaxis_title="Frequenza",
    width=900,
    height=300,
    bargap=0.01,
    margin=dict(l=0, r=0, t=0, b=0),
)

# fig.write_image("errore_percentuale_uomo.png")
fig.show()

In [ ]:
df = df_results[["body_part", "error"]]
df_res = df.groupby(["body_part"]).mean()
fig = px.box(df, x="body_part", y="error")

fig.update_layout(
    yaxis_title="Percentage error",
    xaxis_title="",
    yaxis=dict(ticksuffix="%"),
    width=900,
    height=300,
    margin=dict(l=0, r=0, t=0, b=0),
)

# fig.write_image("error_body_part.png")
fig.show()